# Efficient Yelp API Calls
Author: Kim Hazed Delfino

## Imports

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


## Set API credentials and connection

In [63]:
# Load Credentials
with open('/Users/kim/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [64]:
# Instantiate API 
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api


In [65]:
# Set API call params
LOCATION = "Miami, FL"
TERM = "Dinner Cruise"

In [66]:

# Create path for results 
JSON_FILE = f"Data/results_in_progress_{LOCATION.split(',')[0]}_{TERM.replace(' ','_')}.json"
JSON_FILE

'Data/results_in_progress_Miami_Dinner_Cruise.json'

# Custom Functions to eliminate repeating line of codes

In [67]:
# Text colors for visuals
reset_color = "\033[0m"
red = "\033[0;31;40m[!]" + reset_color
green = "\033[0;32;40m[i]" + reset_color
yellow = "\033[0;33;40m[i]" + reset_color


# Custom function to create/delete JSON file
def json_file_maker(JSON_FILE, delete_if_exist=False):

    # Check if JSON file exists
    file = os.path.isfile(JSON_FILE)

    # If file exists
    if file: 

        # Check if user wants to delete if exists 
        if delete_if_exist:
            print(red+f" {JSON_FILE} Already Exist. Deleting previous file. . . ")
            # Delete prev file
            os.remove(JSON_FILE) 
            # Create new JSON file by calling the same function (recussive)
            json_file_maker(JSON_FILE,delete_if_exist=False)
            print(green+ f" {JSON_FILE} file has been created!")

        else: 
            print(red + f" {JSON_FILE} Already Exist."+reset_color)      

    # If no existing file
    else:

        print(yellow+f" {JSON_FILE} is a new file, Creating empty folder. . . "+reset_color)

        # Get folder name
        folder = os.path.dirname(JSON_FILE)

        if len(folder)>0:
            # Create the folder
            os.makedirs(folder,exist_ok=True)

        with open(JSON_FILE,'w') as f:
            json.dump([],f)
        
        return print(green + f" New JSON file {JSON_FILE} has been created!"+reset_color)

In [68]:
# Create custom fuction to run query 
def run_query(json_file ,location:str, term:str, write_mode = False):

    # Load json file
    with open(json_file,'r') as f:
        previous_results = json.load(f)

    # Set offset based on prev results
    n_results = len(previous_results)
    print(yellow+f"{n_results} previous results")

    # Run query and store in results 
    results = yelp_api.search_query(location=LOCATION,
                                    term= TERM,
                                    offset= n_results)

    total_results = results['total']
    per_page = len(results['businesses'])
    total_pages = math.ceil((total_results-n_results)/per_page)

    if not write_mode:
        return print(green+f"{total_results} total results with {per_page} per page and {total_pages} total pages")
    
    elif write_mode:

        # run query and write the results to json file
        for i in tqdm_notebook(range(1,total_pages+1)):
            # Load json file
            with open(json_file,'r') as f:
                previous_results = json.load(f)

            # Set breakpoint
            limit = 1000
            current = n_results+per_page
            if current > limit:
                print(red+f"Exeeded {limit} API calls limit, Query shutting down.")
                break

            # Set offset based on prev results
            n_results = len(previous_results)
            if i > 1:
                print(yellow+f"{n_results} previous results")

            # Run query and store in results 
            results = yelp_api.search_query(location=LOCATION,
                                            term= TERM,
                                            offset= n_results)
            # Update results
            previous_results.extend(results['businesses'])

            # Save current results to json file
            with open(JSON_FILE,'w') as f:
                json.dump(previous_results,f)
            
            time.sleep(.2)
        
        return print(green+"Query Complete! Final Results: "+"\033[0;32;40m"+f"{results['total']}"+reset_color)

In [69]:
# Create json file 
json_file_maker(JSON_FILE,delete_if_exist=False)

[i] Data/results_in_progress_Miami_Dinner_Cruise.json is a new file, Creating empty folder. . . 
[i] New JSON file Data/results_in_progress_Miami_Dinner_Cruise.json has been created!


In [70]:
# try if our func will catch json file already exists 
json_file_maker(JSON_FILE,delete_if_exist=False)

[!] Data/results_in_progress_Miami_Dinner_Cruise.json Already Exist.


In [71]:
# Run query
run_query(JSON_FILE,LOCATION,TERM)

[i]0 previous results
[i]116 total results with 20 per page and 6 total pages


In [72]:
# Run query and save results into json file 
run_query(JSON_FILE,LOCATION,TERM,write_mode=True)

[i]0 previous results


  0%|          | 0/6 [00:00<?, ?it/s]

[i]20 previous results
[i]40 previous results
[i]60 previous results
[i]80 previous results
[i]100 previous results
[i]Query Complete! Final Results: 116


In [73]:
# Convert JSON to DataFrame
final_df = pd.read_json(JSON_FILE)
display(final_df.sample(10))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
64,CMdjIxBBmNpfMSinbk2a4w,hampton-inn-and-suites-by-hilton-miami-airport...,Hampton Inn & Suites by Hilton Miami Airport S...,https://s3-media2.fl.yelpcdn.com/bphoto/2YoONS...,False,https://www.yelp.com/biz/hampton-inn-and-suite...,86,"[{'alias': 'hotels', 'title': 'Hotels'}]",3.5,"{'latitude': 25.779061016768495, 'longitude': ...",[],"{'address1': '777 NW 57th Ave', 'address2': ''...",+13052625400,(305) 262-5400,6005.594721,$$
96,G4iTsJGURb8s66NeUbn5-w,royal-palm-south-beach-miami-a-tribute-portfol...,"Royal Palm South Beach Miami, a Tribute Portfo...",https://s3-media4.fl.yelpcdn.com/bphoto/A1Qn7j...,False,https://www.yelp.com/biz/royal-palm-south-beac...,502,"[{'alias': 'hotels', 'title': 'Hotels'}]",2.5,"{'latitude': 25.7885094, 'longitude': -80.1293...",[],"{'address1': '1545 Collins Ave', 'address2': '...",+13056045700,(305) 604-5700,9941.713344,$$
35,VkdtCMovs1JMZUKZBinNdw,play-time-water-sports-miami-beach,Play Time Water Sports,https://s3-media2.fl.yelpcdn.com/bphoto/382XVr...,False,https://www.yelp.com/biz/play-time-water-sport...,77,"[{'alias': 'boatcharters', 'title': 'Boat Char...",4.0,"{'latitude': 25.7654593493781, 'longitude': -8...",[],"{'address1': '300 Alton Rd', 'address2': '', '...",+15167786738,(516) 778-6738,4441.069908,NaN
114,msST2WziUttElo2pFGVzwg,voyager-of-the-seas-miami,Voyager of the Seas,https://s3-media2.fl.yelpcdn.com/bphoto/VtgiMm...,False,https://www.yelp.com/biz/voyager-of-the-seas-m...,2,"[{'alias': 'tours', 'title': 'Tours'}, {'alias...",4.0,"{'latitude': 25.7752657, 'longitude': -80.1777...",[],"{'address1': '1050 Carribean Way', 'address2':...",+18002566649,(800) 256-6649,5075.929085,NaN
11,K3IPYmaK-xRlH24zNrOmcg,carnival-breeze-miami-2,Carnival Breeze,https://s3-media2.fl.yelpcdn.com/bphoto/RplWMk...,False,https://www.yelp.com/biz/carnival-breeze-miami...,70,"[{'alias': 'travelservices', 'title': 'Travel ...",3.5,"{'latitude': 25.775279, 'longitude': -80.167577}",[],"{'address1': 'N Cruise Blvd', 'address2': '', ...",+18003279501,(800) 327-9501,6138.281616,NaN
2,9DSfS-_P3jUKdIO-cNhDoQ,tikki-beach-charters-miami-5,Tikki Beach Charters,https://s3-media4.fl.yelpcdn.com/bphoto/VxRIpS...,False,https://www.yelp.com/biz/tikki-beach-charters-...,6,"[{'alias': 'boatcharters', 'title': 'Boat Char...",5.0,"{'latitude': 25.7950053, 'longitude': -80.1872...",[],"{'address1': '1900 N Bayshore Dr', 'address2':...",+13053502112,(305) 350-2112,4341.181095,NaN
95,tjlxvkk-uBS6CV8rvzOY5g,juliet-sailing-and-diving-miami,Juliet Sailing and Diving,https://s3-media2.fl.yelpcdn.com/bphoto/211YQ6...,False,https://www.yelp.com/biz/juliet-sailing-and-di...,10,"[{'alias': 'scuba', 'title': 'Scuba Diving'}, ...",5.0,"{'latitude': 25.7738321, 'longitude': -80.1899...",[],"{'address1': '25 SE 2nd Ave', 'address2': 'Ste...",+18665585438,(866) 558-5438,3961.580112,NaN
71,pjgVbc_CjeU8t8nb1_AlDQ,gray-line-miami-miami,Gray Line Miami,https://s3-media1.fl.yelpcdn.com/bphoto/Z1pAbY...,False,https://www.yelp.com/biz/gray-line-miami-miami...,25,"[{'alias': 'tours', 'title': 'Tours'}]",1.5,"{'latitude': 25.77447, 'longitude': -80.19054}",[],"{'address1': '199 E Flagler St', 'address2': '...",+18776431258,(877) 643-1258,3886.854539,NaN
70,-jZnCMibPIXC3rY1CpSJVw,carnival-victory-downtown-miami,Carnival Victory,https://s3-media3.fl.yelpcdn.com/bphoto/wxuyeG...,False,https://www.yelp.com/biz/carnival-victory-down...,148,"[{'alias': 'boatcharters', 'title': 'Boat Char...",3.0,"{'latitude': 25.7766517566255, 'longitude': -8...",[],"{'address1': 'Panama Way', 'address2': '', 'ad...",+18007647419,(800) 764-7419,5866.640281,NaN
99,nao14PM9SXsLQkkv9eGl_g,bruschi-boat-rental-miami,Bruschi Boat Rental,https://s3-media4.fl.yelpcdn.com/bphoto/6X91MT...,False,https://www.yelp.com/biz/bruschi-boat-rental-m...,52,"[{'alias': 'boatcharters', 'title': 'Boat Char...",4.5,"{'latitude': 25.77481, 'longitude': -80

In [74]:
# Check for duplicates
final_df.duplicated(subset='id').sum()

0

In [75]:
s = "qweqwe 123"
s.replace(" ", "_")

'qweqwe_123'

In [76]:
# Save df to as CSV file
final_df.to_csv(f'Data/final_results_{LOCATION.split(",")[0]}_{TERM.replace(" ", "_")}.csv',index=False)